# 📊 6강: 성적 분석 & 위험 감지

> 성적 CSV를 업로드하면 AI가 추이 분석 + 위험 학생 감지를 수행합니다.
> **pandas + matplotlib + Gemini** 조합!

In [ ]:
!pip install -q google-genai
from google import genai
from google.colab import userdata
import getpass, pandas as pd
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['font.family'] = 'DejaVu Sans'

try:
    API_KEY = userdata.get('GEMINI_API_KEY')
except:
    API_KEY = getpass.getpass('🔑 Gemini API Key: ')

client = genai.Client(api_key=API_KEY)
MODEL = 'gemini-2.5-flash'
print('✅ 준비 완료!')

In [ ]:
# 📊 샘플 성적 데이터 (또는 파일 업로드)
data = {
    '이름': ['김민준','이서연','박지호','최수아','정예준','한소율','김도현','이하은','박서준','윤지아'],
    '3월': [78,92,45,85,60,70,50,88,65,82],
    '4월': [82,90,50,83,65,72,48,90,62,85],
    '5월': [85,88,48,82,62,75,45,92,60,88],
    '6월': [80,91,42,80,68,78,52,89,58,90],
    '7월': [75,93,38,78,72,80,55,91,55,87],
    '8월': [70,95,35,76,75,82,58,94,52,85]
}
df = pd.DataFrame(data)
print('📋 성적 데이터:')
df

## 🔬 실습 1: 성적 추이 시각화

In [ ]:
# 📈 성적 추이 차트
months = ['3월','4월','5월','6월','7월','8월']

fig, ax = plt.subplots(figsize=(12, 6))
for _, row in df.iterrows():
    scores = [row[m] for m in months]
    color = 'red' if scores[-1] - scores[0] < -5 else 'green' if scores[-1] - scores[0] > 5 else 'gray'
    ax.plot(months, scores, marker='o', label=row['이름'], linewidth=2,
            color=color, alpha=0.8)

ax.set_ylabel('Score', fontsize=14)
ax.set_title('Student Score Trends (Red=Declining, Green=Improving)', fontsize=16)
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax.grid(True, alpha=0.3)
ax.set_ylim(20, 100)
plt.tight_layout()
plt.savefig('score_trend.png', dpi=150, bbox_inches='tight')
plt.show()
print('📊 빨간색 = 하락 | 초록색 = 상승 | 회색 = 유지')

## 🔬 실습 2: 위험 학생 자동 감지

In [ ]:
# 🚨 위험 학생 감지 (3개월 연속 하락)
df['변화_최근3개월'] = df['8월'] - df['6월']
df['변화_전체'] = df['8월'] - df['3월']
df['위험'] = df.apply(
    lambda r: '🔴 위험' if r['변화_최근3개월'] < -5 else
              '🟡 주의' if r['변화_전체'] < 0 else
              '🟢 양호', axis=1)

# 결과 표시
result = df[['이름', '3월', '8월', '변화_전체', '변화_최근3개월', '위험']]
result = result.sort_values('변화_전체')
print('🚨 위험 학생 감지 결과:')
print('='*60)
result

## 🔬 실습 3: AI가 분석 리포트 생성

In [ ]:
# 🤖 AI 성적 분석 리포트
risk_students = result[result['위험'].str.contains('위험|주의')]

prompt = f"""
너는 교육 데이터 분석가이자 학원 경영 컨설턴트야.

아래 성적 데이터를 분석해서 리포트를 작성해줘.

전체 데이터:
{result.to_string()}

리포트 구성:
1. 📊 전체 요약 (평균, 추세)
2. 🏆 Most Improved 학생 (칭찬 포인트)
3. 🚨 위험 학생 분석 (원인 추정 + 대응 전략)
4. 📋 학급 전체 권장 액션 3가지
5. 💡 학부모 개별 연락 우선순위 (상/중/하)

데이터 기반으로만 분석. 추측 최소화.
"""

response = client.models.generate_content(model=MODEL, contents=prompt)
report = response.text
print('📊 AI 성적 분석 리포트')
print('='*60)
print(report)

## 🔬 실습 4: 위험 학생별 개입 메시지

In [ ]:
# 위험/주의 학생에게 보낼 메시지 생성
for _, s in risk_students.iterrows():
    prompt = f"""
{s['이름']} 학생의 성적이 {s['변화_전체']:+d}점 변동했습니다.
3월: {s['3월']}점 → 8월: {s['8월']}점

이 학부모에게 보낼 카톡 메시지 (5줄, 이모지):
- 걱정이 아닌 관심의 톤
- 구체적 대응 방안 1가지 제안
- 상담 예약 유도
"""
    response = client.models.generate_content(model=MODEL, contents=prompt)
    print(f'\n📱 {s["이름"]} ({s["위험"]}) 학부모 메시지:')
    print('-'*40)
    print(response.text)

In [ ]:
# 💾 저장
output = f"성적분석 리포트:\n{report}"
with open('성적분석_리포트.txt', 'w', encoding='utf-8') as f:
    f.write(output)
from google.colab import files
files.download('성적분석_리포트.txt')
files.download('score_trend.png')
print('✅ 리포트 + 차트 다운로드!')

## 🏆 6강 완료!
👉 [7강: 퇴원생 관리 & 재등록](https://colab.research.google.com/github/Reasonofmoon/cedu-1/blob/main/notebooks/07_re_enroll.ipynb)